# Pipeline de Chamada de Variantes do SARS-CoV-2

## Objetivo
Reconstruir o genoma viral presente na amostra e identificar mutações em relação ao genoma de referência, determinando a linhagem.

## Passos do Pipeline
1. Alinhamento ao Genoma de Referência (BWA-MEM)
2. Processamento do BAM (samtools, remoção de duplicatas)
3. Chamada de Variantes (bcftools)
4. Anotação de Variantes (SnpEff)
5. Determinação de Linhagem (Pangolin)

In [7]:
# Configuração inicial
import os
import sys
from pathlib import Path

# Obter diretório raiz do projeto (assumindo que notebook está em notebooks/)
notebook_dir = Path().resolve()  # Diretório atual de trabalho
if 'notebooks' in str(notebook_dir):
    # Se estamos em notebooks/, subir 1 nível
    project_root = notebook_dir.parent
else:
    # Se executando de outro lugar, tentar encontrar a raiz
    # Ou usar caminho absoluto se necessário
    project_root = Path('/Users/larissa/Desktop/TCC_metatrascriptomica')

# Adicionar scripts ao path
sys.path.insert(0, str(project_root))

# Verificar se scripts existe
scripts_dir = project_root / "scripts"
if not scripts_dir.exists():
    raise FileNotFoundError(
        f"Diretório 'scripts' não encontrado em {project_root}.\n"
        f"Diretório atual de trabalho: {Path().resolve()}\n"
        f"Tente executar: os.chdir('{project_root / 'notebooks'}')"
    )

# Configurar caminhos relativos
DATA_DIR = project_root / "data"
PROCESSED_DIR = DATA_DIR / "processed"
REFERENCES_DIR = DATA_DIR / "references"
RESULTS_DIR = project_root / "results"
VARIANTS_DIR = RESULTS_DIR / "variants"

# Criar diretórios
for directory in [VARIANTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Mudar para diretório do notebook (opcional, mas útil)
os.chdir(project_root / "notebooks")

print(f"Project root: {project_root}")
print(f"Data directory: {DATA_DIR}")
print(f"Scripts directory: {scripts_dir} {'✅' if scripts_dir.exists() else '❌'}")
print(f"Diretório de trabalho: {os.getcwd()}")

Project root: /Users/larissa/Desktop/TCC_metatrascriptomica
Data directory: /Users/larissa/Desktop/TCC_metatrascriptomica/data
Scripts directory: /Users/larissa/Desktop/TCC_metatrascriptomica/scripts ✅
Diretório de trabalho: /Users/larissa/Desktop/TCC_metatrascriptomica/notebooks


## 1. Alinhamento ao Genoma de Referência

In [8]:
from scripts.variant_calling import align_reads_bwa, sam_to_bam
import os

# Arquivos de entrada (reads sem hospedeiro)
input_r1 = str(PROCESSED_DIR / "patient_joao_nonhuman_R1.fastq.gz")
input_r2 = str(PROCESSED_DIR / "patient_joao_nonhuman_R2.fastq.gz")

# Genoma de referência SARS-CoV-2
reference_fasta = str(REFERENCES_DIR / "NC_045512.2.fasta")

# Verificar arquivos de entrada
if not os.path.exists(input_r1):
    print(f"⚠️ Arquivo R1 não encontrado: {input_r1}")
if not os.path.exists(input_r2):
    print(f"⚠️ Arquivo R2 não encontrado: {input_r2}")
if not os.path.exists(reference_fasta):
    print(f"⚠️ Genoma de referência não encontrado: {reference_fasta}")

# Alinhar reads
output_sam = str(VARIANTS_DIR / "patient_joao_aligned.sam")
if os.path.exists(input_r1) and os.path.exists(input_r2) and os.path.exists(reference_fasta):
    print("Executando alinhamento com BWA-MEM (isso pode demorar vários minutos)...")
    aligned_sam = align_reads_bwa(
        input_r1=input_r1,
        input_r2=input_r2,
        reference_fasta=reference_fasta,
        output_sam=output_sam,
        threads=8
    )
    print("Alinhamento concluído!")

    # Converter SAM para BAM ordenado
    sorted_bam = str(VARIANTS_DIR / "patient_joao_sorted.bam")
    if os.path.exists(output_sam):
        print("Convertendo SAM para BAM e ordenando...")
        bam_file = sam_to_bam(sorted_bam=sorted_bam, sam_file=output_sam)
        print("BAM processado!")
else:
    print("❌ Não é possível executar alinhamento: arquivos necessários não encontrados.")
    sorted_bam = None

Executando alinhamento com BWA-MEM (isso pode demorar vários minutos)...
Alinhando reads com BWA-MEM...
Alinhamento concluído: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_aligned.sam
Alinhamento concluído!
Convertendo SAM para BAM e ordenando...
Convertendo SAM para BAM...
Ordenando BAM...
Indexando BAM...
BAM processado: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_sorted.bam
BAM processado!


## 2. Chamada de Variantes

In [9]:
from scripts.variant_calling import call_variants_bcftools

# Chamar variantes
output_vcf = str(VARIANTS_DIR / "patient_joao_variants.vcf.gz")

if sorted_bam and os.path.exists(sorted_bam) and os.path.exists(reference_fasta):
    print("Chamando variantes com bcftools (isso pode demorar alguns minutos)...")
    vcf_file = call_variants_bcftools(
        bam_file=sorted_bam,
        reference_fasta=reference_fasta,
        output_vcf=output_vcf
    )
    print("Chamada de variantes concluída!")
else:
    print("❌ Não é possível chamar variantes: BAM ou referência não encontrados.")
    output_vcf = None

Chamando variantes com bcftools (isso pode demorar alguns minutos)...
Chamando variantes com bcftools...
Variantes chamadas: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_variants.vcf.gz
Chamada de variantes concluída!


## 3. Anotação de Variantes

In [10]:
from scripts.variant_annotation import annotate_variants_snpeff, parse_annotated_vcf, extract_spike_mutations
import pandas as pd

# Anotar variantes
annotated_vcf = str(VARIANTS_DIR / "patient_joao_variants_annotated.vcf")

if output_vcf and os.path.exists(output_vcf):
    print("Anotando variantes com SnpEff...")
    annotated = annotate_variants_snpeff(
        input_vcf=output_vcf,
        output_vcf=annotated_vcf,
        genome="NC_045512.2"
    )
    print("Anotação concluída!")

    # Parsear variantes anotadas
    if os.path.exists(annotated_vcf):
        print("\n📊 Parseando variantes anotadas...")
        variants_df = parse_annotated_vcf(annotated_vcf)
        print(f"Total de variantes: {len(variants_df)}")

        # Extrair mutações no gene Spike
        spike_mutations = extract_spike_mutations(variants_df)
        print(f"Mutações no gene Spike: {len(spike_mutations)}")
        
        if len(spike_mutations) > 0:
            print("\nMutações no Gene Spike:")
            print(spike_mutations[['POS', 'REF', 'ALT', 'EFFECT', 'IMPACT']].to_string(index=False))

        # Salvar tabela
        variants_table = str(VARIANTS_DIR / "patient_joao_variants_table.csv")
        variants_df.to_csv(variants_table, index=False)
        print(f"\n✅ Tabela de variantes salva em: {variants_table}")
else:
    print("❌ Não é possível anotar variantes: arquivo VCF não encontrado.")

Anotando variantes com SnpEff...
Anotando variantes com SnpEff...
Variantes anotadas: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_variants_annotated.vcf
Anotação concluída!

📊 Parseando variantes anotadas...
Total de variantes: 20
Mutações no gene Spike: 3

Mutações no Gene Spike:
  POS REF ALT           EFFECT   IMPACT
21629   C   A missense_variant MODERATE
22331   G   A missense_variant MODERATE
23403   A   G missense_variant MODERATE

✅ Tabela de variantes salva em: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_variants_table.csv


## 4. Determinação de Linhagem

In [12]:
from scripts.lineage_determination import determine_lineage_pangolin, generate_consensus_fasta

# Gerar genoma consenso (alternativa: usar BAM diretamente)
consensus_fasta = str(VARIANTS_DIR / "patient_joao_consensus.fasta")

# Determinar linhagem
lineage_csv = str(VARIANTS_DIR / "patient_joao_lineage.csv")

if sorted_bam and os.path.exists(sorted_bam):
    # Tentar determinar linhagem diretamente do BAM
    print("Determinando linhagem com Pangolin (usando BAM)...")
    try:
        lineage_df = determine_lineage_pangolin(
            input_file=sorted_bam,
            output_csv=lineage_csv,
            input_type="bam"
        )
        if lineage_df is not None and len(lineage_df) > 0:
            lineage_name = lineage_df.iloc[0].get('lineage', 'N/A')
            print(f"\n✅ Linhagem identificada: {lineage_name}")
            print("\nDetalhes completos:")
            print(lineage_df.to_string(index=False))
    except Exception as e:
        print(f"⚠️ Erro ao determinar linhagem com BAM: {e}")
        print("Tentando gerar genoma consenso primeiro...")
        
        # Gerar genoma consenso e tentar novamente
        if os.path.exists(reference_fasta):
            consensus = generate_consensus_fasta(
                bam_file=sorted_bam,
                reference_fasta=reference_fasta,
                output_fasta=consensus_fasta
            )
            
            if os.path.exists(consensus_fasta):
                print("Determinando linhagem com Pangolin (usando consenso FASTA)...")
                lineage_df = determine_lineage_pangolin(
                    input_file=consensus_fasta,
                    output_csv=lineage_csv,
                    input_type="fasta"
                )
                if lineage_df is not None and len(lineage_df) > 0:
                    lineage_name = lineage_df.iloc[0].get('lineage', 'N/A')
                    print(f"\n✅ Linhagem identificada: {lineage_name}")
else:
    print("❌ Não é possível determinar linhagem: BAM não encontrado.")

Determinando linhagem com Pangolin (usando BAM)...
Determinando linhagem com Pangolin...
  Input: patient_joao_sorted.bam
  Type: bam
⚠️ Erro ao determinar linhagem com BAM: Pangolin falhou: /opt/miniconda3/envs/tcc-metatranscriptomica/lib/python3.10/site-packages/pangolin/utils/data_checks.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Error: the input query fasta could not be parsed.
Double check your query fasta and that compressed stdin was not passed.
Please enter your fasta sequence file and refer to pangolin usage at: https://cov-lineages.org/pangolin.html for detailed instructions.

Tentando gerar genoma consenso primeiro...
Gerando genoma consenso...
Genoma consenso gerado: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_consensu